In [1]:
from pathlib import Path
from io import StringIO
import pandas as pd

In [2]:
grid, moves = Path("data/15sample2.txt").read_text().split("\n\n")
grid = (
    pd.read_csv(StringIO(grid), engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(how="all", axis=1)
    .values
)
moves = [x for l in moves.split("\n") for x in l]